# Ustawienia środowiska symulacji

W tym notebooku ustawiamy takie rzeczy jak data, pogoda, model Ziemi itp. Każda sekcja jest odpowiednio opisana i sformatowana, aby ułatwić korzystanie z pliku.

## Ustawienie czasu


In [ ]:
# Wyświetlenie dostępnych stref czasowych
import pytz

print(pytz.all_timezones)

In [ ]:
# Przykład ustawienia środowiska z określoną datą i strefą czasową
from rocketpy import Environment

env = Environment(
    date=(2025, 12, 18, 12),  # rok, miesiąc, dzień, godzina
    timezone="Europe/Warsaw"
)

### Ustawienie daty na jutro przy użyciu `datetime`

In [ ]:
from datetime import date, timedelta

tomorrow = date.today() + timedelta(days=1)

date_info = (tomorrow.year, tomorrow.month, tomorrow.day, 9)  # Godzina w czasie UTC

print("Data jutrzejsza:", date_info)

## Ustawienie lokalizacji startu rakiety


In [ ]:
# Przykład ustawienia lokalizacji startu
from rocketpy import Environment

env = Environment()

env.set_location(latitude=32.988528, longitude=-106.975056)

env.prints.launch_site_details()

**Przykładowe lokalizacje do testów:**

- Far-out: `latitude=35.35`, `longitude=-117.81`

- Toruń: `latitude=52.97`, `longitude=18.57`

### Wysokość nad poziomem morza

In [ ]:
# Ustawienie wysokości nad poziomem morza (jeśli znamy)
from rocketpy import Environment

env = Environment(elevation=110)

In [ ]:
# Możemy też ustawić wysokość później
env.set_elevation(120)

In [ ]:
# Przykład pobrania wysokości z Open-Elevation API (jeśli nie znamy)
env = Environment(
    date=(2019, 2, 10, 18),
    latitude=49.179577383781776,  # mniej więcej jak leżą Rysy
    longitude=20.088008357667263
)
env.set_elevation("Open-Elevation")

## Model atmosfery

Ostatnią i najważniejszą rzeczą do ustawienia jest model atmosfery. Mamy do wyboru:

1. **Standardowy** - International Standard Atmosphere (ISA)
2. **Własny** - definiujemy parametry ręcznie
3. **Prognozy** - dane z prognoz pogody (GFS, GEFS)
4. **Reanaliza** - dane historyczne (ERA5)
5. **Ensemble** - zbiór prognoz do analizy Monte Carlo

### Model standardowy

> **Uwaga:** Międzynarodowa Standardowa Atmosfera (ISA) ma poprawnie określone tylko profile temperatury i ciśnienia. Inne profile można wyprowadzić, jednak wiatry są automatycznie ustawione na 0 m/s.

Przydatne, jeśli chcemy mieć powtarzalne warunki do testów.

In [ ]:
# Inicjalizacja środowiska z modelem standardowym
from rocketpy import Environment

env = Environment()  # Domyślnie ustawia International Standard Atmosphere

env.plots.atmospheric_model()

### Własny model atmosfery

In [ ]:
# Przykład własnego modelu atmosfery
from rocketpy import Environment

env_custom_atm = Environment()

In [ ]:
env_custom_atm.set_atmospheric_model(
    type="custom_atmosphere",
    pressure=None,
    temperature=300,
    wind_u=[
        (0, 5),    # 5 m/s na 0 m
        (1000, 10) # 10 m/s na 1000 m
    ],
    wind_v=[
        (0, -2),   # -2 m/s na 0 m
        (500, 3),  # 3 m/s na 500 m
        (1600, 2), # 2 m/s na 1600 m
    ],
)

env_custom_atm.plots.atmospheric_model()

Możemy dowolnie ustalać parametry atmosfery. Więcej informacji: [RocketPy Custom Atmosphere Documentation](https://docs.rocketpy.org/en/latest/user/environment/1-atm-models/custom_atmosphere.html)

### Prognoza pogody (forecast)

Jeśli chcemy zasymulować warunki, jakie będą panować podczas testu rakiety, możemy użyć prognozy.

In [ ]:
from datetime import datetime, timedelta
from rocketpy import Environment

tomorrow = datetime.now() + timedelta(days=1)

env_gfs = Environment(date=tomorrow, latitude=80.179577383781776, longitude=20.088008357667263)

env_gfs.set_atmospheric_model(type="forecast", file="GFS")

env_gfs.plots.atmospheric_model()

Więcej informacji o dostawcach prognoz: [RocketPy Forecast Documentation](https://docs.rocketpy.org/en/latest/user/environment/1-atm-models/forecast.html)

### Dane historyczne (reanaliza)

Jeśli chcemy użyć danych pogodowych z przeszłości, możemy wykorzystać reanalysis, np. ERA5.

### Model Ensemble

Ensemble to zbiór wielu prognoz, przydatny szczególnie do analizy Monte Carlo i oceny niepewności prognoz.

In [ ]:
# Przykład użycia ensemble (GEFS)
from datetime import date, timedelta
from rocketpy import Environment

tomorrow = date.today() + timedelta(days=1)

env_gefs = Environment(
    date=(tomorrow.year, tomorrow.month, tomorrow.day, 18),
    latitude=-21.960641,
    longitude=-47.482122,
    elevation=640,
)
env_gefs.set_atmospheric_model(type="Ensemble", file="GEFS")

In [ ]:
# Wybór konkretnego członka ensemble (np. 3. prognozy)
env_gefs.select_ensemble_member(3)

In [ ]:
# Wyświetlenie wszystkich informacji o środowisku
env_gefs.all_info()

## Dodatkowe opcje

### Model Ziemi
Klasa `Environment` w RocketPy obecnie wspiera tylko model płaskiej Ziemi. Możliwe jest jednak ustawienie własnej wartości grawitacji, jeśli jest to wymagane do specjalistycznych symulacji.

### Topografia terenu
Istnieje również opcja ustawienia topografii terenu, co może być przydatne przy testach w górzystych regionach.